## Parte 1: Recolección de información
#### Web Scraping a la "Infobae" para investigación de informes referentes a riegos pre electorales

In [1]:
# Importación de librerias
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import random

# Opciones del navegador
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--headless')  
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

In [2]:
# Clase para personalización
class bcolors:
    WARNING = '\033[93m'
    FAIL = '\033[91m'

# Control de errores
try:   
    # Definición de función
    def Seconds(result):
        hours = round(result/3600)
        minutes = round((result - (hours * 3600))/60)
        seconds = round(result - ((hours * 3600) + (minutes * 60)))
        if seconds < 0: seconds = seconds*-1
        if minutes < 0: minutes = minutes*-1
        final_time = "El sitema tardó: " + str(hours) + " horas " + str(minutes) + " minutos " + str(seconds) + " segundos en finalizar su ejeción"
        return final_time
    
except Exception as e: 
    print(bcolors.FAIL + "Revisa el error => ", e) # Impresión del error personalizado

In [ ]:
# Definición de variables
headers = []
headers_text = []
headers_link = []
all_news = []

# Captura del tema de interés de la persona
interest = input("¿Cuál es el riesgo pre electoral de su interés?: ")

# Mensaje para visualización del inicio del proceso
print("Se inició el Web Scraping a 'Infobae' del tema: '" + interest + "', por favor espere...")

# Variable para definir el inicio de tiempo en ejecución
start = time.time()

# Acceso a la página
driver = webdriver.Chrome(
    service = Service(ChromeDriverManager().install()),
    options = chrome_options
)

# Página principal
driver.get("https://www.infobae.com/colombia/")

# Simulación de comportamiento humano
time.sleep(random.randrange(10,15))

# Menú (página principal)
menu = driver.find_element(By.XPATH, '//*[@id="siteheader"]/div[2]/span')
menu.click();

# Simulación de comportamiento humano
time.sleep(random.randrange(5,10))

# Menú (sidebar)
menu_button = driver.find_element(By.XPATH, '//*[@id="queryly-label"]/button')
menu_button.click();

# Simulación de comportamiento humano
time.sleep(random.randrange(5,10))

# Campo de búsqueda (ventana emergente)
search_bar = driver.find_element(By.XPATH, '//*[@id="queryly_query"]')
search_bar.clear()
search_bar.send_keys(interest)

# Simulación de comportamiento humano
time.sleep(random.randrange(5,10))

# Envío de consulta (Enter)
search_bar.send_keys(Keys.RETURN)

# Simulación de comportamiento humano
time.sleep(random.randrange(5,10))

# Aplicación de filtro (noticias de "Colombia")
filter_button = driver.find_element(By.XPATH, '//*[@id="section_filter"]/div[4]/a')
filter_button.click();

# Simulación de comportamiento humano
time.sleep(random.randrange(5,10))

# Cantida de resultados encontrados
results = driver.find_element(By.XPATH, '//*[@id="resultdata"]/div[1]/div')

# Simulación de comportamiento humano
time.sleep(random.randrange(5,10))


# Ciclo para recorrer todas las páginas
while(True):
    
    # Div (contiene las noticias o artículos)
    elements = driver.find_elements(By.CSS_SELECTOR, "div.queryly_item_row")
    
    # Ciclo para separa cada uno de los elementos
    for element in elements:
    # ****************************************************************************************************************
        headers =  element.find_elements(By.TAG_NAME, 'a')
        if headers: 
            for header in headers:
                headers_text.append(header.text)
                headers_link.append(header.get_attribute('href'))
        else: 
            headers_text.append('...')
            headers_link.append('...')
    # ****************************************************************************************************************

    # Construcción del diccionario 
    news = {
        "headers_text": headers_text,
        "headers_link": headers_link
    }
    
    time.sleep(random.randrange(3,5))
    # Control del error al momento de finalizar el páginado
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'a.next_btn')
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        if next_button: 
            elements = []
            # Simulación de comportamiento humano
            time.sleep(random.randrange(3,5))
            next_button.click()
        else: break;
    except Exception as e: 
        break
    except NoSuchElementException:
        break
    except TimeoutException:
        break
    
    # Construcción inicial del dataframe
    df = pd.DataFrame(news)
    all_news.append(df)
    
# Contrucción final del DataFrame
df = pd.concat(all_news)

# Limpieza del DataFrame
df = df.drop_duplicates()

# Cerrar el controlador de Chrome
driver.quit()

In [ ]:
df